In [32]:
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf
import keyboard

In [33]:
mp_hands = mp.solutions.hands
hands=mp_hands.Hands(max_num_hands=1,min_detection_confidence=0.7)
mp_drawing = mp.solutions.drawing_utils


In [34]:
# Initialize the video capture
cap = cv2.VideoCapture(0)


In [35]:
    
while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the frame with MediaPipe Hands
        results = hands.process(rgb_frame)
        if results.multi_hand_landmarks:                     #it means that the MediaPipe Hands model has detected one or more hands in the frame
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    frame,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2),
                    mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2)
                )
                
                # Detecting hand state

                wrist = hand_landmarks.landmark[mp_hands.HandLandmark.WRIST]

                index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
                middle_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP]
                ring_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP]
                pinky_tip = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP]


                index_finger_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP]
                middle_finger_mcp = [(index_finger_mcp.x + ring_finger_tip.x) / 2, (index_finger_mcp.y + ring_finger_tip.y) / 2]
                ring_finger_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_MCP]
                pinky_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_MCP]
                
                

                
                # Determine if the palm is open or closed
                if (index_finger_tip.y < index_finger_mcp.y and
                    middle_finger_tip.y < middle_finger_mcp[1] and
                    ring_finger_tip.y < ring_finger_mcp.y and
                    pinky_tip.y < pinky_mcp.y):
                    cv2.putText(frame, 'Palm Open', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2, cv2.LINE_AA)
                    # keyboard.press_and_release('w')
                else:
                    cv2.putText(frame, 'Palm Closed', (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                    # keyboard.press_and_release('s')


                # Detecting hand turn (left or right)
                if index_finger_tip.x < wrist.x and pinky_tip.x > wrist.x:
                    cv2.putText(frame, 'Palm Turn Left', (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)
                    
                elif index_finger_tip.x > wrist.x and pinky_tip.x < wrist.x:
                    cv2.putText(frame, 'Palm Turn Right', (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 255), 2, cv2.LINE_AA)

        # Display the frame
        cv2.imshow('MediaPipe Hand Gesture', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

In [36]:
cap.release()
cv2.destroyAllWindows()